### Import Required Libraries and Set Up Environment Variables

In [84]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json

In [131]:
# Set environment variables from the .env in the local environment
load_dotenv('environment.env')

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")

print(type(nyt_api_key))


<class 'str'>


### Access the New York Times API

In [86]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"


# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'
# filter_query = 'blockbuster'
# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
# https://api.nytimes.com/svc/search/v2/articlesearch.json?q=election&api-key=yourkey
# https://developer.nytimes.com/docs/articlesearch-product/1/routes/articlesearch.json/get

query_url = f"{url}&fq={filter_query}&api-key={nyt_api_key}&begin_date={begin_date}&end_date={end_date}"


In [87]:
# Make a Request and check response 
response = requests.get(query_url)
response

<Response [200]>

In [88]:

reviews_list = []           # list to store the reviews
review_count = 0;           
for page in range(0, 3):    # loop through pages 0-19
# create query with a page number; API results show 10 articles at a time
    query_url_by_page = f"{query_url}&page={str(page)}"

# Make a "GET" request and Retrieve Articles (retrieve the JSON)
    try:
# Request Reviews by page using URL Query above
        reviews = requests.get(query_url_by_page).json()    
# Loop through the response & save/append each article to the reviews_list
        for review in reviews["response"]["docs"]:
            reviews_list.append(review)
            review_count += 1
# Print page that was just retrieved
        print(f'Successfully obtained {review_count} reviews, page {page}')
# If there is page number w/ no results, print page # and break from loop
    except:
        print(f'Failed to obtain reviews, page {page}')
        break

# Add 12 sec b/w queries to stay within API query limits
    time.sleep(12)

Successfully obtained 10 reviews, page 0
Successfully obtained 20 reviews, page 1
Successfully obtained 30 reviews, page 2


In [97]:
#  Print sample of reviews added to reviews_list
count = 0
print(f'Total reviews obtained: {len(reviews_list)}' )
for r in reviews_list:
    count +=1
    print(f"Sample review ({count}): {r['headline']['main']}") #snippet
    if count == 3:
        break

Total reviews obtained: 30
Sample review (1): Review: ‘Kangaroo: A Love-Hate Story’ Exposes a Wildlife Massacre
Sample review (2): Love’s Eternal Masquerade
Sample review (3): ‘The Tomorrow Man’ Review: Love Among the Neuroses


In [98]:
# Preview first 5 results in JSON format using json.dumps w/ argument indent=4 to format data
for i in range(5):
    print(f"\nJSON Sample {i} ({reviews_list[i]['headline']['main']})") #snippet
    print(json.dumps(reviews_list[i], indent=4))


JSON Sample 0 (Review: ‘Kangaroo: A Love-Hate Story’ Exposes a Wildlife Massacre)
{
    "abstract": "The documentary looks at the mass killings of kangaroos for pet-food companies, leather processors and ranchers in Australia.",
    "web_url": "https://www.nytimes.com/2018/01/18/movies/kangaroo-a-love-hate-story-review.html",
    "snippet": "The documentary looks at the mass killings of kangaroos for pet-food companies, leather processors and ranchers in Australia.",
    "lead_paragraph": "Brutality overrides everything else in \u201cKangaroo: A Love-Hate Story\u201d \u2014 views of beautiful landscapes are hard to enjoy after seeing wild animals, including their young, maimed and slaughtered. But the filmmakers are determined to sound a wake-up siren, and they blast it here with extra strength.",
    "print_section": "C",
    "print_page": "9",
    "source": "The New York Times",
    "multimedia": [
        {
            "rank": 0,
            "subtype": "xlarge",
            "captio

In [109]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
reviews_df = pd.json_normalize(reviews_list)
reviews_df.head(10)

,abstract,web_url,snippet,lead_paragraph,print_section,print_page,source,multimedia,keywords,pub_date,...,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,The documentary looks at the mass killings of ...,https://www.nytimes.com/2018/01/18/movies/kang...,The documentary looks at the mass killings of ...,Brutality overrides everything else in “Kangar...,C,9,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...","[{'name': 'creative_works', 'value': 'Kangaroo...",2018-01-18T12:00:23+0000,...,Review: ‘Kangaroo: A Love-Hate Story’ Exposes ...,None,None,Kangaroo: A Love-Hate Story,None,None,None,By Ken Jaworowski,"[{'firstname': 'Ken', 'middlename': None, 'las...",None
1,"In “Jewtopia,” a young man asks a childhood fr...",https://www.nytimes.com/2013/09/20/movies/jewt...,"In “Jewtopia,” a young man asks a childhood fr...",Did you hear the one about the gentile and the...,C,16,The New York Times,"[{'rank': 0, 'subtype': 'wide', 'caption': Non...","[{'name': 'subject', 'value': 'Movies', 'rank'...",2013-09-19T23:33:15+0000,...,Love’s Eternal Masquerade,Movie Review,None,Jewtopia,None,None,None,By David DeWitt,"[{'firstname': 'David', 'middlename': None, 'l...",None
2,Blythe Danner and John Lithgow strain to eleva...,https://www.nytimes.com/2019/05/21/movies/the-...,Blythe Danner and John Lithgow strain to eleva...,Sometimes you’d think there was a conspiracy a...,C,4,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...","[{'name': 'subject', 'value': 'Movies', 'rank'...",2019-05-21T11:00:01+0000,...,‘The Tomorrow Man’ Review: Love Among the Neur...,None,None,"They Whirl, They Twirl, They Tango",None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
3,Adam Driver and Scarlett Johansson self-consci...,https://www.nytimes.com/2019/11/05/movies/marr...,Adam Driver and Scarlett Johansson self-consci...,"Traditionally, a story that ends in matrimony ...",C,1,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...","[{'name': 'subject', 'value': 'Movies', 'rank'...",2019-11-05T19:01:02+0000,...,‘Marriage Story’ Review: Dance Me to the End o...,critic’s pick,None,"Friendly Split, Shattering Break",None,None,None,By A.O. Scott,"[{'firstname': 'A.', 'middlename': 'O.', 'last...",None
4,Luca Guadagnino’s latest stars Timothée Chalam...,https://www.nytimes.com/2022/11/17/movies/bone...,Luca Guadagnino’s latest stars Timothée Chalam...,Anyone who travels the roads of America must s...,C,8,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...","[{'name': 'subject', 'value': 'Movies', 'rank'...",2022-11-17T12:00:06+0000,...,‘Bones and All’ Review: You Eat What You Are,None,None,"It’s Eat, Prey, Love on a Journey of Self-Disc...",None,None,None,By A.O. Scott,"[{'firstname': 'A.', 'middlename': 'O.', 'last...",None
5,"Anton Yelchin, who played Chekov in the J.J. A...",https://www.nytimes.com/2019/08/08/movies/love...,"Anton Yelchin, who played Chekov in the J.J. A...","The death of the actor Anton Yelchin, in a fre...",C,8,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...","[{'name': 'subject', 'value': 'Documentary Fil...",2019-08-08T09:00:10+0000,...,"‘Love, Antosha’ Review: A Heartbreaking Look a...",Critic’s Pick,None,"Love, Antosha",None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None
6,Religion comes between two girls falling in lo...,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,"In “You Can Live Forever,” Jaime and Marike do...",C,9,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...","[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,...,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 

In [110]:
reviews_df.columns

Index(['abstract', 'web_url', 'snippet', 'lead_paragraph', 'print_section',
       'print_page', 'source', 'multimedia', 'keywords', 'pub_date',
       'document_type', 'news_desk', 'section_name', 'type_of_material', '_id',
       'word_count', 'uri', 'headline.main', 'headline.kicker',
       'headline.content_kicker', 'headline.print_headline', 'headline.name',
       'headline.seo', 'headline.sub', 'byline.original', 'byline.person',
       'byline.organization'],
      dtype='object')

In [111]:
print(reviews_df.iloc[0]['headline.main'])

reviews_edit_df = reviews_df.copy()
reviews_edit_df['title'] = reviews_edit_df['headline.main']
reviews_edit_df['title'].head()

Review: ‘Kangaroo: A Love-Hate Story’ Exposes a Wildlife Massacre


0    Review: ‘Kangaroo: A Love-Hate Story’ Exposes ...
1                            Love’s Eternal Masquerade
2    ‘The Tomorrow Man’ Review: Love Among the Neur...
3    ‘Marriage Story’ Review: Dance Me to the End o...
4         ‘Bones and All’ Review: You Eat What You Are
Name: title, dtype: object

In [126]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
#   Title is between unicode characters \u2018 and \u2019. 
#   End string should include " Review" to avoid cutting title early

# str : str[start, stop]
# str.find(sub[, start[, end]] )  # sub: substring to be searched in the string.

reviews_edit_df['title'] = reviews_edit_df['headline.main'].apply(\
        lambda str: str[str.find("\u2018")+1 : str.find("\u2019 Review")])
reviews_edit_df['title'].head()



0    Kangaroo: A Love-Hate Story’ Exposes a Wildlif...
1                             Love’s Eternal Masquerad
2                                     The Tomorrow Man
3                                       Marriage Story
4                                        Bones and All
Name: title, dtype: object

In [127]:
reviews_edit_df['title'].head()

0    Kangaroo: A Love-Hate Story’ Exposes a Wildlif...
1                             Love’s Eternal Masquerad
2                                     The Tomorrow Man
3                                       Marriage Story
4                                        Bones and All
Name: title, dtype: object

In [128]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # print(f'item {item}')
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string:
#   Use `extract_keywords` function to convert the `"keywords"` column from 
#     a list of dictionaries to strings using the `apply()` method.

reviews_edit_02_df = reviews_edit_df.copy()
reviews_edit_02_df['keywords'] = reviews_edit_02_df['keywords'].apply(extract_keywords)
reviews_edit_02_df.head()
        

,abstract,web_url,snippet,lead_paragraph,print_section,print_page,source,multimedia,keywords,pub_date,...,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,The documentary looks at the mass killings of ...,https://www.nytimes.com/2018/01/18/movies/kang...,The documentary looks at the mass killings of ...,Brutality overrides everything else in “Kangar...,C,9,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...",creative_works: Kangaroo: A Love-Hate Story (M...,2018-01-18T12:00:23+0000,...,None,None,Kangaroo: A Love-Hate Story,None,None,None,By Ken Jaworowski,"[{'firstname': 'Ken', 'middlename': None, 'las...",None,Kangaroo: A Love-Hate Story’ Exposes a Wildlif...
1,"In “Jewtopia,” a young man asks a childhood fr...",https://www.nytimes.com/2013/09/20/movies/jewt...,"In “Jewtopia,” a young man asks a childhood fr...",Did you hear the one about the gentile and the...,C,16,The New York Times,"[{'rank': 0, 'subtype': 'wide', 'caption': Non...","subject: Movies;persons: Sergei, Ivan;creative...",2013-09-19T23:33:15+0000,...,Movie Review,None,Jewtopia,None,None,None,By David DeWitt,"[{'firstname': 'David', 'middlename': None, 'l...",None,Love’s Eternal Masquerad
2,Blythe Danner and John Lithgow strain to eleva...,https://www.nytimes.com/2019/05/21/movies/the-...,Blythe Danner and John Lithgow strain to eleva...,Sometimes you’d think there was a conspiracy a...,C,4,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...",subject: Movies;creative_works: The Tomorrow M...,2019-05-21T11:00:01+0000,...,None,None,"They Whirl, They Twirl, They Tango",None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Tomorrow Man
3,Adam Driver and Scarlett Johansson self-consci...,https://www.nytimes.com/2019/11/05/movies/marr...,Adam Driver and Scarlett Johansson self-consci...,"Traditionally, a story that ends in matrimony ...",C,1,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...","subject: Movies;persons: Baumbach, Noah;creati...",2019-11-05T19:01:02+0000,...,critic’s pick,None,"Friendly Split, Shattering Break",None,None,None,By A.O. Scott,"[{'firstname': 'A.', 'middlename': 'O.', 'last...",None,Marriage Story
4,Luca Guadagnino’s latest stars Timothée Chalam...,https://www.nytimes.com/2022/11/17/movies/bone...,Luca Guadagnino’s latest stars Timothée Chalam...,Anyone who travels the roads of America must s...,C,8,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...",subject: Movies;creative_works: Bones and All ...,2022-11-17T12:00:06+0000,...,None,None,"It’s Eat, Prey, Love on a Journey of Self-Disc...",None,None,None,By A.O. Scott,"[{'firstname': 'A.', 'middlename': 'O.', 'last...",None,Bones and All


In [139]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
titles = reviews_edit_02_df['title'].to_list()
titles

['Kangaroo: A Love-Hate Story’ Exposes a Wildlife Massacr',
 'Love’s Eternal Masquerad',
 'The Tomorrow Man',
 'Marriage Story',
 'Bones and All',
 'Love, Antosha',
 'You Can Live Forever',
 'Prem Ratan Dhan Payo,’ a Bollywood Tale of a Prince-and-Plebe Doubl',
 'A Promoter Finds a New Cause: Go',
 'Now They’ll Love Me, a Twin Sister Scheme',
 'They’ll Love Me When I’m Dead’ Documents Orson Welles’s Last Fil',
 'A Journal for Jordan',
 'A United Kingdom’ With Love That Tested Racial Toleranc',
 'Sophie and the Rising Sun’ Has Forbidden Love in a Foreboding Tim',
 'Passengers’ Trapped on a Spaceship Find Love Amid Despai',
 'Undine',
 'Waiting for Bojangles',
 'Love and Unhappiness, in Soft Shades of Gra',
 'Lost in Paris',
 'Lost and Love’ (‘Shi Gu’) Stars Andy Lau as a Man Searching for His Lost So',
 'From This Day Forward’ Attests to Love’s Adaptabilit',
 'How He Fell in Love,’ an Extramarital Bond Unfold',
 'Young Love, Interrupted by a Nuclear Bom',
 'Love the Coopers,’ a Dysfunct

### Access The Movie Database API

In [136]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
print(type(tmdb_api_key))
tmdb_key_string = "&api_key=" + tmdb_api_key


<class 'str'>


NameError: name 'query' is not defined

In [144]:

# Create an empty list called `tmdb_movies_list` to store results from API requests. 
# This will contain a list of dictionaries.
tmdb_movies_list = []

# * Create a variable called `request_counter`; initialize it with the value of `1`. 
request_counter = 1

# Loop through the titles list created from the movie reviews DataFrame
for t in titles:

    if request_counter == 5:
        break

# Check if we need to sleep before making a request
# Create request counter to sleep the requests after multiple of 50 requests
# Use `time.sleep(1)` when it reaches a multiple of 50
    if request_counter % 50 == 0:

# Print a message to indicate that the application is sleeping.
        print(f'request counter: {request_counter}, commence sleep.')
        time.sleep(1)

#   Use a `try` clause
    try:
       
# Perform a "GET" request for The Movie Database, request full movie details
#  Use movie query (starting with url), send title to The Movie Database search.
        query = t
        query_url = f"{url}{query}{tmdb_key_string}"
        print(query_url)
# retrieve the full movie details in JSON format.
        req = requests.get(query_url_by_page).json()  
        



# Increment by one every time you iterate through the `titles` list.
# Add 1 to the request counter
        request_counter += 1
        
#   Use the except clause to print out a statement if a movie is not found.
    # Use the except clause to print out a statement if a movie
    # is not found.
    except:
        print(f'Failed to obtain Movie ID in TMDB for title {t}')
        break    





https://api.themoviedb.org/3/search/movie?query=Kangaroo: A Love-Hate Story’ Exposes a Wildlife Massacr&api_key=a78f3a912736e51a4277ddf596af9b2f
<Response [200]>
https://api.themoviedb.org/3/search/movie?query=Love’s Eternal Masquerad&api_key=a78f3a912736e51a4277ddf596af9b2f
<Response [200]>
https://api.themoviedb.org/3/search/movie?query=The Tomorrow Man&api_key=a78f3a912736e51a4277ddf596af9b2f
<Response [200]>
https://api.themoviedb.org/3/search/movie?query=Marriage Story&api_key=a78f3a912736e51a4277ddf596af9b2f
<Response [200]>


In [ ]:
#   Collect the movie ID from the first result.


#  Extract the genre names from the results into a list called genres.
        genres[] = 

#  Extract the spoken_languages' English name from results to list spoken_languages.
        spoken_languages[]=

#  Extract the production_countries' name from results to list production_countries.
        production_countries = 

# Add the relevant data to a dictionary:
#  Create a dictionary with the following results: 
#      `title`, `original_title`, `budget`, `original_language`, 
#      `homepage`, `overview`, `popularity`, `runtime`, `revenue`, 
#      `release_date`, `vote_average`, `vote_count`, as well as the 
#      `genres`, `spoken_languages`, and `production_countries` lists you just created

        movie_dictionary = {   'title': , 
                                'original_title': , 
                                'budget': , 
                                'original_language': , 
                                'homepage': , 
                                'overview': , 
                                'popularity': , 
                                'runtime': , 
                                'revenue': , 
                                'release_date': , 
                                'vote_average': , 
                                'vote_count': , 
                                'genres': genres, 
                                'spoken_languages': spoken_languages , 
                                'production_countries': production_countries }

#  # append dictionary to `tmdb_movies_list`
        tmdb_movies_list.append()

#  Print out the name of the movie and a message to indicate that the title was found.
#  Print out the title that was found



      

In [ ]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data


# Preview first 5 results in JSON format using json.dumps w/ argument indent=4 to format data
for i in range(5):
    print(f"\nJSON Sample {i} ({reviews_list[i]['headline']['main']})") #snippet
    print(json.dumps(reviews_list[i], indent=4))

[
    {
        "title": "The Attachment Diaries",
        "original_title": "El apego",
        "budget": 0,
        "genre": [
            "Drama",
            "Mystery",
            "Thriller",
            "Horror"
        ],
        "language": "es",
        "spoken_languages": [
            "Spanish"
        ],
        "homepage": "",
        "overview": "Argentina, 1970s. A desperate young woman goes to a clinic to have a clandestine abortion. As her pregnancy is already through the fourth month, the doctor refuses. Instead, she proposes to sell the baby to one of her clients and offers to provide shelter in her house until the child is born. Their disturbed personalities will become intertwined in a strange and dangerous relationship.",
        "popularity": 0.708,
        "runtime": 102,
        "revenue": 0,
        "release_date": "2021-10-07",
        "vote_average": 4.0,
        "vote_count": 2,
        "production_countries": [
            "Argentina"
        ]
    },
    

In [ ]:
# Convert the results to a DataFrame


,title,original_title,budget,genre,language,spoken_languages,homepage,overview,popularity,runtime,revenue,release_date,vote_average,vote_count,production_countries
0,The Attachment Diaries,El apego,0,"[Drama, Mystery, Thriller, Horror]",es,[Spanish],,"Argentina, 1970s. A desperate young woman goes...",0.708,102,0,2021-10-07,4.000,2,[Argentina]
1,You Can Live Forever,You Can Live Forever,0,"[Drama, Romance]",en,"[French, English]",https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",12.027,96,0,2023-03-24,7.643,21,[Canada]
2,A Tourist's Guide to Love,A Tourist's Guide to Love,0,"[Romance, Comedy, Adventure]",en,[English],https://www.netflix.com/title/81424906,"After an unexpected break up, a travel executi...",25.369,96,0,2023-04-21,6.389,122,[United States of America]
3,Other People's Children,Les Enfants des autres,0,[Drama],fr,[French],https://www.wildbunch.biz/movie/other-peoples-...,"Rachel loves her life, her students, her frien...",16.246,104,0,2022-09-21,6.900,137,[France]
4,One True Loves,One True Loves,0,"[Romance, Drama, Comedy]",en,[English],,Emma and Jesse are living the perfect life tog...,28.872,100,37820,2023-04-07,6.681,36,"[Germany, United States of America]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150,The Last Face,The Last Face,0,[Drama],en,"[English, Portuguese]",,"Miguel, a heroic Spanish doctor, puts himself ...",9.081,130,0,2017-01-11,5.617,184,[United States of America]
151,Lost in Paris,Paris pieds nus,0,[Comedy],fr,[French],,Fiona visits Paris for the first time to assis...,6.798,83,0,2017-01-14,6.051,108,"[Belgium, France]"
152,The Other Half,The Other Half,0,"[Drama, Comedy, Romance]",en,[English],,When English Mark surprises his American bride...,1.058,94,0,2006-05-15,6.708,12,[United Kingdom]
153,The Ottoman Lieutenant,The Ottoman Lieutenant,40000000,"[Romance, Drama, War]",en,"[English, Turkish]",https://www.uphe.com/movies/the-ottoman-lieute...,"Lillie, a determined American woman, ventures ...",12.097,111,413844,2017-03-10,6.150,236,"[Turkey, United States of America]"


### Merge and Clean the Data for Export

In [ ]:
# Merge the New York Times reviews and TMDB DataFrames on title


,title,original_title,budget,genre,language,spoken_languages,homepage,overview,popularity,runtime,...,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,The Attachment Diaries,El apego,0,"[Drama, Mystery, Thriller, Horror]",es,[Spanish],,"Argentina, 1970s. A desperate young woman goes...",0.708,102,...,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,You Can Live Forever,You Can Live Forever,0,"[Drama, Romance]",en,"[French, English]",https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",12.027,96,...,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
2,One True Loves,One True Loves,0,"[Romance, Drama, Comedy]",en,[English],,Emma and Jesse are living the perfect life tog...,28.872,100,...,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,"[{'firstname': 'Brandon', 'middlename': None, ...",None
3,The Lost Weekend: A Love Story,The Lost Weekend: A Love Story,0,[Documentary],en,[],,May Pang lovingly recounts her life in rock & ...,2.368,97,...,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None
4,A Thousand and One,A Thousand and One,0,[Drama],en,[English],https://www.focusfeatures.com/a-thousand-and-one/,Struggling but unapologetically living on her ...,11.704,116,...,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,In Search of Fellini,In Search of Fellini,0,"[Drama, Adventure]",en,[English],,"Lucy, a small-town girl from Ohio, discovers t...",4.103,93,...,Review: Love of Il Maestro Drives ‘In Search o...,None,None,In Search of Fellini,None,None,None,By Monica Castillo,"[{'firstname': 'Monica', 'middlename': None, '...",None
121,The Last Face,The Last Face,0,[Drama],en,"[English, Portuguese]",,"Miguel, a heroic Spanish doctor, puts himself ...",9.081,130,...,Review: Aid Workers in Love and War in Sean Pe...,None,None,The Last Face,None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None
122,Lost in Paris,Paris pieds nus,0,[Comedy],fr,[French],,Fiona visits Paris for the first time to assis...,6.798,83,...,Review: Finding Love (and Slapstick) While ‘Lo...,None,None,Lost in Paris,None,None,None,By Ben Kenigsberg,"[{'firstname': 'Ben', 'middlename': None, 'las...",None
123,The Other Half,The Other Half,0,"[Drama, Comedy, Romance]",en,[English],,When English Mark surprises his American bride...,1.058,94,...,Review: A Combustible Pair Find Love in ‘The O...,None,None,Review: A Combustible Pair Find Love in ‘The O...,None,None,None,By Andy Webster,"[{'firstname': 'Andy', 'middlename': None, 'la...",None


In [ ]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing


# Create a list of characters to remove


# Loop through the list of columns to fix

    # Convert the column to type 'str'


    # Loop through characters to remove


# Display the fixed DataFrame


,title,original_title,budget,genre,language,spoken_languages,homepage,overview,popularity,runtime,...,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,The Attachment Diaries,El apego,0,"Drama, Mystery, Thriller, Horror",es,Spanish,,"Argentina, 1970s. A desperate young woman goes...",0.708,102,...,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,You Can Live Forever,You Can Live Forever,0,"Drama, Romance",en,"French, English",https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",12.027,96,...,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
2,One True Loves,One True Loves,0,"Romance, Drama, Comedy",en,English,,Emma and Jesse are living the perfect life tog...,28.872,100,...,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,"[{'firstname': 'Brandon', 'middlename': None, ...",None
3,The Lost Weekend: A Love Story,The Lost Weekend: A Love Story,0,Documentary,en,,,May Pang lovingly recounts her life in rock & ...,2.368,97,...,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None
4,A Thousand and One,A Thousand and One,0,Drama,en,English,https://www.focusfeatures.com/a-thousand-and-one/,Struggling but unapologetically living on her ...,11.704,116,...,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None


In [ ]:
# Drop "byline.person" column


In [ ]:
# Delete duplicate rows and reset index


,title,original_title,budget,genre,language,spoken_languages,homepage,overview,popularity,runtime,...,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.organization
0,The Attachment Diaries,El apego,0,"Drama, Mystery, Thriller, Horror",es,Spanish,,"Argentina, 1970s. A desperate young woman goes...",0.708,102,...,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,None
1,You Can Live Forever,You Can Live Forever,0,"Drama, Romance",en,"French, English",https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",12.027,96,...,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,None
2,One True Loves,One True Loves,0,"Romance, Drama, Comedy",en,English,,Emma and Jesse are living the perfect life tog...,28.872,100,...,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,None
3,The Lost Weekend: A Love Story,The Lost Weekend: A Love Story,0,Documentary,en,,,May Pang lovingly recounts her life in rock & ...,2.368,97,...,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,None,None,None,By Glenn Kenny,None
4,A Thousand and One,A Thousand and One,0,Drama,en,English,https://www.focusfeatures.com/a-thousand-and-one/,Struggling but unapologetically living on her ...,11.704,116,...,971,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,None,None,None,By Manohla Dargis,None


In [ ]:
# Export data to CSV without the index
